#Imports


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import os
import torchvision
import pandas as pd
from torchvision.io import read_image
from torchvision.utils import save_image
#import Image
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.optim as optim
import cv2
from sklearn.metrics import accuracy_score

import torchvision.transforms.functional as fn
import torch.nn as nn
import torch.nn.functional as F

#Data Exploration

In [ ]:
#dataset_path = '/content/gdrive/MyDrive/NeedleImages/'
labels_path = '/content/gdrive/MyDrive/NeedleImages/Labelsc.csv'
img_labels = pd.read_csv(labels_path)
labels = pd.Series(np.where(img_labels['Label'] == 'yes', 1, 0),img_labels.index)
#print(labels)
print("images in class 1:", labels.value_counts()[1])
print("images in class 0:", labels.value_counts()[0])

images in class 1: 441
images in class 0: 191


#Dataloader

In [ ]:
# Dataloader

class MedDataset_new(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        # print(self.img_labels.shape)
        # print(self.img_labels)

        self.img_labels['Label'] = pd.Series(np.where(self.img_labels['Label'] == 'yes', 1, 0),
          self.img_labels.index)
       # self.im
        # print(self.img_labels)
        #print( self.img_labels.iloc[0, 2])
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        #img_path = os.path.join(self.img_dir, ('n'+self.img_labels.iloc[idx, 1]))
        img_path = os.path.join(self.img_dir, (self.img_labels.iloc[idx, 1]))
        image = read_image(img_path)

       # image  = torch.cat((image, image, image), 0)
        #print(image.shape)
        # if resize:
        image = fn.resize(image,128)
        label = self.img_labels.iloc[idx, 2]

        #add contrast
        #image = fn.adjust_contrast(image, 6)
        # print(label )

        #print(image.shape)
        #plt.imshow(image[0])
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        # plt.imshow(image[0])
        # print(label)
        return image, label

In [ ]:
dataset_path = '/content/gdrive/MyDrive/NeedleImages/'

#dataset_path = '/content/gdrive/MyDrive/NeedleImages/imgs1/'
# dataset = MedDataset(os.path.join(dataset_path,'Labels.csv'), dataset_path)
dataset = MedDataset_new('/content/gdrive/MyDrive/NeedleImages/Labelsc.csv', dataset_path)
generator1 = torch.Generator().manual_seed(22)

train_set, test_set = torch.utils.data.random_split(dataset, [505,127])

trainloader = DataLoader(dataset = train_set, batch_size = 64, shuffle = True, generator=generator1)
testloader = DataLoader(dataset= test_set, batch_size = 64, shuffle = True, generator=generator1)



#DeepCNN

In [ ]:
class DeepCNN(nn.Module):

    def __init__(self):
        super().__init__()

        # onvolutional layers (3,16,32)
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 16, kernel_size=(5, 5), stride=2, padding=1)
        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size=(5, 5), stride=2, padding=1)
        self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size=(3, 3), padding=1)

        # conected layers
        self.fc1 = nn.Linear(in_features= 576, out_features=500)
        self.fc2 = nn.Linear(in_features=500, out_features=50)
        self.fc3 = nn.Linear(in_features=50, out_features=2)


    def forward(self, X):

        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2)

        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2)

        X = F.relu(self.conv3(X))
        X = F.max_pool2d(X, 2)

        X = X.view(X.shape[0], -1)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)

        return X

In [ ]:
model = DeepCNN()

losses = []
accuracies = []
epoches = 50
start = time.time()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

# Model Training...
for epoch in range(epoches):

    epoch_loss = 0
    epoch_accuracy = 0

    for X, y in trainloader:

        preds = model(X.float())
        loss = loss_fn(preds, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        accuracy = ((preds.argmax(dim=1) == y).float().mean())
        epoch_accuracy += accuracy
        epoch_loss += loss
        print('.', end='', flush=True)

    epoch_accuracy = epoch_accuracy/len(trainloader)
    accuracies.append(epoch_accuracy)
    epoch_loss = epoch_loss / len(trainloader)
    losses.append(epoch_loss)

    print("\n --- Epoch: {}, train loss: {:.4f}, train acc: {:.4f}, time: {}".format(epoch, epoch_loss, epoch_accuracy, time.time() - start))

    # test set accuracy
    with torch.no_grad():

        test_epoch_loss = 0
        test_epoch_accuracy = 0

        for test_X, test_y in testloader:

            test_preds = model(test_X.float())
            test_loss = loss_fn(test_preds, test_y)

            test_epoch_loss += test_loss
            test_accuracy = ((test_preds.argmax(dim=1) == test_y).float().mean())
            test_epoch_accuracy += test_accuracy

        test_epoch_accuracy = test_epoch_accuracy/len(testloader)
        test_epoch_loss = test_epoch_loss / len(testloader)

        print("Epoch: {}, test loss: {:.4f}, test acc: {:.4f}, time: {}\n".format(epoch, test_epoch_loss, test_epoch_accuracy, time.time() - start))

........
 --- Epoch: 0, train loss: 2.4305, train acc: 0.5728, time: 2.8549582958221436
Epoch: 0, test loss: 0.6872, test acc: 0.5200, time: 3.3820252418518066

........
 --- Epoch: 1, train loss: 0.6368, train acc: 0.6527, time: 6.226945877075195
Epoch: 1, test loss: 0.5803, test acc: 0.7329, time: 6.774516820907593

........
 --- Epoch: 2, train loss: 0.5821, train acc: 0.6859, time: 9.665087699890137
Epoch: 2, test loss: 0.5362, test acc: 0.7475, time: 10.22899866104126

........
 --- Epoch: 3, train loss: 0.5481, train acc: 0.7140, time: 14.116719007492065
Epoch: 3, test loss: 0.5797, test acc: 0.6615, time: 14.777974367141724

........
 --- Epoch: 4, train loss: 0.5236, train acc: 0.7393, time: 17.667240381240845
Epoch: 4, test loss: 0.5433, test acc: 0.6928, time: 18.19721269607544

........
 --- Epoch: 5, train loss: 0.4632, train acc: 0.7625, time: 21.022440671920776
Epoch: 5, test loss: 0.5368, test acc: 0.6772, time: 21.574347734451294

........
 --- Epoch: 6, train loss: 0.3

KeyboardInterrupt: ignored

#CNN

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super().__init__()

        # onvolutional layers (3,16,32)
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 16, kernel_size=(5, 5), stride=2, padding=1)
        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size=(5, 5), stride=2, padding=1)
        self.conv3 = nn.Conv2d(in_channels = 16, out_channels = 64, kernel_size=(3, 3), padding=1)

        # conected layers
        self.fc1 = nn.Linear(in_features= 14400, out_features=500)
        self.fc2 = nn.Linear(in_features=500, out_features=50)
        self.fc3 = nn.Linear(in_features=50, out_features=2)


    def forward(self, X):

        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2)

        # X = F.relu(self.conv2(X))
        # X = F.max_pool2d(X, 2)

        X = F.relu(self.conv3(X))
        X = F.max_pool2d(X, 2)

        X = X.view(X.shape[0], -1)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)

        return X

In [ ]:
model = Net()

losses = []
accuracies = []
epoches = 50
start = time.time()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

# Model Training...
for epoch in range(epoches):

    epoch_loss = 0
    epoch_accuracy = 0

    for X, y in trainloader:

        preds = model(X.float())
        loss = loss_fn(preds, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        accuracy = ((preds.argmax(dim=1) == y).float().mean())
        epoch_accuracy += accuracy
        epoch_loss += loss
        print('.', end='', flush=True)

    epoch_accuracy = epoch_accuracy/len(trainloader)
    accuracies.append(epoch_accuracy)
    epoch_loss = epoch_loss / len(trainloader)
    losses.append(epoch_loss)

    print("\n --- Epoch: {}, train loss: {:.4f}, train acc: {:.4f}, time: {}".format(epoch, epoch_loss, epoch_accuracy, time.time() - start))

    # test set accuracy
    with torch.no_grad():

        test_epoch_loss = 0
        test_epoch_accuracy = 0

        for test_X, test_y in testloader:

            test_preds = model(test_X.float())
            test_loss = loss_fn(test_preds, test_y)

            test_epoch_loss += test_loss
            test_accuracy = ((test_preds.argmax(dim=1) == test_y).float().mean())
            test_epoch_accuracy += test_accuracy

        test_epoch_accuracy = test_epoch_accuracy/len(testloader)
        test_epoch_loss = test_epoch_loss / len(testloader)

        print("Epoch: {}, test loss: {:.4f}, test acc: {:.4f}, time: {}\n".format(epoch, test_epoch_loss, test_epoch_accuracy, time.time() - start))

........
 --- Epoch: 0, train loss: 67.9368, train acc: 0.5977, time: 8.157029390335083
Epoch: 0, test loss: 1.7022, test acc: 0.3074, time: 8.809293985366821

........
 --- Epoch: 1, train loss: 2.4448, train acc: 0.5726, time: 13.477833032608032
Epoch: 1, test loss: 0.4954, test acc: 0.7561, time: 14.317841053009033

........
 --- Epoch: 2, train loss: 0.8982, train acc: 0.6314, time: 20.09014630317688
Epoch: 2, test loss: 0.7579, test acc: 0.7323, time: 20.93763780593872

........
 --- Epoch: 3, train loss: 0.5734, train acc: 0.7266, time: 25.062905311584473
Epoch: 3, test loss: 0.4675, test acc: 0.7796, time: 25.68881058692932

........
 --- Epoch: 4, train loss: 0.4335, train acc: 0.7789, time: 30.980095863342285
Epoch: 4, test loss: 0.4730, test acc: 0.7638, time: 31.686641931533813

........
 --- Epoch: 5, train loss: 0.3843, train acc: 0.8509, time: 35.67477369308472
Epoch: 5, test loss: 0.4872, test acc: 0.7715, time: 36.322324991226196

........
 --- Epoch: 6, train loss: 0.3

KeyboardInterrupt: ignored

In [ ]:
print(preds.argmax(dim=1), y)
print(test_preds.argmax(dim=1), test_y)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0,
        1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
        1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1]) tensor([1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
        0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1,
        0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1])
tensor([1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
        0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1]) tensor([1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1,
        1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1,
        0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1])


In [ ]:
c1 = 0
c2 = 0
print(len(y))
for i in range(len(test_y)):

  if test_preds.argmax(dim=1)[i]== test_y[i]:
    c1 = c1+1
  else:
    print(preds.argmax(dim=1)[i], y[i])
    c2 = c2+1
print(c1,c2)
accuracy = ((test_preds.argmax(dim=1) == test_y).float().mean())
print(accuracy)
#len(trainloader)

64
tensor(1) tensor(0)
tensor(1) tensor(0)
tensor(1) tensor(0)
tensor(0) tensor(1)
tensor(1) tensor(1)
tensor(0) tensor(1)
tensor(1) tensor(0)
tensor(0) tensor(0)
tensor(1) tensor(0)
tensor(1) tensor(1)
tensor(0) tensor(1)
tensor(0) tensor(0)
tensor(1) tensor(0)
tensor(0) tensor(1)
tensor(1) tensor(0)
tensor(0) tensor(1)
tensor(0) tensor(0)
46 17
tensor(0.7302)


#arch3: Alexnet

In [ ]:

class AlexNet(nn.Module):
    def __init__(self, num_classes=2):
        super(AlexNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 96, kernel_size=11, stride=4, padding=0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(1024, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [ ]:
model = AlexNet()

losses = []
accuracies = []
epoches = 50
start = time.time()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

# Model Training...
for epoch in range(epoches):

    epoch_loss = 0
    epoch_accuracy = 0

    for X, y in trainloader:

        preds = model(X.float())
        loss = loss_fn(preds.squeeze(1), y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        accuracy = ((preds.argmax(dim=1) == y).float().mean())
        epoch_accuracy += accuracy
        epoch_loss += loss
        print('.', end='', flush=True)

    epoch_accuracy = epoch_accuracy/len(trainloader)
    accuracies.append(epoch_accuracy)
    epoch_loss = epoch_loss / len(trainloader)
    losses.append(epoch_loss)

    print("\n --- Epoch: {}, train loss: {:.4f}, train acc: {:.4f}, time: {}".format(epoch, epoch_loss, epoch_accuracy, time.time() - start))

    # test set accuracy
    with torch.no_grad():

        test_epoch_loss = 0
        test_epoch_accuracy = 0

        for test_X, test_y in testloader:

            test_preds = model(test_X.float())
            test_loss = loss_fn(test_preds, test_y)

            test_epoch_loss += test_loss
            test_accuracy = ((test_preds.argmax(dim=1) == test_y).float().mean())
            test_epoch_accuracy += test_accuracy

        test_epoch_accuracy = test_epoch_accuracy/len(testloader)
        test_epoch_loss = test_epoch_loss / len(testloader)

        print("Epoch: {}, test loss: {:.4f}, test acc: {:.4f}, time: {}\n".format(epoch, test_epoch_loss, test_epoch_accuracy, time.time() - start))

........
 --- Epoch: 0, train loss: 3.4500, train acc: 0.5294, time: 24.726656675338745
Epoch: 0, test loss: 0.6107, test acc: 0.7319, time: 26.583229541778564

........
 --- Epoch: 1, train loss: 0.6246, train acc: 0.6895, time: 48.54720687866211
Epoch: 1, test loss: 0.5584, test acc: 0.7326, time: 51.42438292503357

........
 --- Epoch: 2, train loss: 0.5844, train acc: 0.6895, time: 73.0847795009613
Epoch: 2, test loss: 0.5435, test acc: 0.7320, time: 74.8574652671814

........
 --- Epoch: 3, train loss: 0.5649, train acc: 0.6898, time: 98.0824248790741
Epoch: 3, test loss: 0.5443, test acc: 0.7320, time: 99.86289930343628

........
 --- Epoch: 4, train loss: 0.5351, train acc: 0.6883, time: 122.41669464111328
Epoch: 4, test loss: 0.5282, test acc: 0.7323, time: 125.28435754776001

........
 --- Epoch: 5, train loss: 0.5299, train acc: 0.6956, time: 147.34710097312927
Epoch: 5, test loss: 0.5458, test acc: 0.7242, time: 149.25779914855957

........
 --- Epoch: 6, train loss: 0.5116,

#arch4 : alexnet modified

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=2):
        super(AlexNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 96, kernel_size=11, stride=4, padding=0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(1024, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        # out = self.layer4(out)
        out = self.layer5(out)
        out = out.reshape(out.size(0), -1)
        #out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [ ]:
model = AlexNet()

losses = []
accuracies = []
epoches = 50
start = time.time()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

# Model Training...
for epoch in range(epoches):

    epoch_loss = 0
    epoch_accuracy = 0

    for X, y in trainloader:

        preds = model(X.float())
        loss = loss_fn(preds.squeeze(1), y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        accuracy = ((preds.argmax(dim=1) == y).float().mean())
        epoch_accuracy += accuracy
        epoch_loss += loss
        print('.', end='', flush=True)

    epoch_accuracy = epoch_accuracy/len(trainloader)
    accuracies.append(epoch_accuracy)
    epoch_loss = epoch_loss / len(trainloader)
    losses.append(epoch_loss)

    print("\n --- Epoch: {}, train loss: {:.4f}, train acc: {:.4f}, time: {}".format(epoch, epoch_loss, epoch_accuracy, time.time() - start))

    # test set accuracy
    with torch.no_grad():

        test_epoch_loss = 0
        test_epoch_accuracy = 0

        for test_X, test_y in testloader:

            test_preds = model(test_X.float())
            test_loss = loss_fn(test_preds, test_y)

            test_epoch_loss += test_loss
            test_accuracy = ((test_preds.argmax(dim=1) == test_y).float().mean())
            test_epoch_accuracy += test_accuracy

        test_epoch_accuracy = test_epoch_accuracy/len(testloader)
        test_epoch_loss = test_epoch_loss / len(testloader)

        print("Epoch: {}, test loss: {:.4f}, test acc: {:.4f}, time: {}\n".format(epoch, test_epoch_loss, test_epoch_accuracy, time.time() - start))

........
 --- Epoch: 0, train loss: 2.6843, train acc: 0.5399, time: 215.02845931053162
Epoch: 0, test loss: 0.6588, test acc: 0.7326, time: 272.28511118888855

........
 --- Epoch: 1, train loss: 0.6787, train acc: 0.6900, time: 291.14488911628723
Epoch: 1, test loss: 0.5909, test acc: 0.7324, time: 293.2492868900299

........
 --- Epoch: 2, train loss: 0.6259, train acc: 0.6888, time: 310.86167430877686
Epoch: 2, test loss: 0.5773, test acc: 0.7325, time: 312.5045199394226

........
 --- Epoch: 3, train loss: 0.5912, train acc: 0.6905, time: 330.15379905700684
Epoch: 3, test loss: 0.5265, test acc: 0.7319, time: 331.6910607814789

........
 --- Epoch: 4, train loss: 0.5954, train acc: 0.6886, time: 351.28659105300903
Epoch: 4, test loss: 0.5270, test acc: 0.7316, time: 353.69235587120056

........
 --- Epoch: 5, train loss: 0.5866, train acc: 0.6851, time: 371.9740309715271
Epoch: 5, test loss: 0.5350, test acc: 0.7476, time: 373.617463350296

........
 --- Epoch: 6, train loss: 0.56

#arch5: alexnet modified

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=2):
        super(AlexNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 96, kernel_size=11, stride=4, padding=0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer5 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(1024, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        #out = self.layer3(out)
        # out = self.layer4(out)
        out = self.layer5(out)
        out = out.reshape(out.size(0), -1)
        #out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [ ]:
model = AlexNet()

losses = []
accuracies = []
epoches = 50
start = time.time()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

# Model Training...
for epoch in range(epoches):

    epoch_loss = 0
    epoch_accuracy = 0

    for X, y in trainloader:

        preds = model(X.float())
        loss = loss_fn(preds.squeeze(1), y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        accuracy = ((preds.argmax(dim=1) == y).float().mean())
        epoch_accuracy += accuracy
        epoch_loss += loss
        print('.', end='', flush=True)

    epoch_accuracy = epoch_accuracy/len(trainloader)
    accuracies.append(epoch_accuracy)
    epoch_loss = epoch_loss / len(trainloader)
    losses.append(epoch_loss)

    print("\n --- Epoch: {}, train loss: {:.4f}, train acc: {:.4f}, time: {}".format(epoch, epoch_loss, epoch_accuracy, time.time() - start))

    # test set accuracy
    with torch.no_grad():

        test_epoch_loss = 0
        test_epoch_accuracy = 0

        for test_X, test_y in testloader:

            test_preds = model(test_X.float())
            test_loss = loss_fn(test_preds, test_y)

            test_epoch_loss += test_loss
            test_accuracy = ((test_preds.argmax(dim=1) == test_y).float().mean())
            test_epoch_accuracy += test_accuracy

        test_epoch_accuracy = test_epoch_accuracy/len(testloader)
        test_epoch_loss = test_epoch_loss / len(testloader)

        print("Epoch: {}, test loss: {:.4f}, test acc: {:.4f}, time: {}\n".format(epoch, test_epoch_loss, test_epoch_accuracy, time.time() - start))

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


........
 --- Epoch: 0, train loss: 2.3157, train acc: 0.5389, time: 16.511329650878906
Epoch: 0, test loss: 0.6209, test acc: 0.6687, time: 17.82980728149414

........
 --- Epoch: 1, train loss: 0.6079, train acc: 0.7059, time: 33.89907622337341
Epoch: 1, test loss: 0.6120, test acc: 0.6693, time: 35.51865267753601

........
 --- Epoch: 2, train loss: 0.5855, train acc: 0.7042, time: 53.77096724510193
Epoch: 2, test loss: 0.6015, test acc: 0.6695, time: 55.45494985580444

........
 --- Epoch: 3, train loss: 0.5701, train acc: 0.7047, time: 70.64759516716003
Epoch: 3, test loss: 0.6022, test acc: 0.6694, time: 71.96150708198547

........
 --- Epoch: 4, train loss: 0.5606, train acc: 0.7047, time: 87.63208818435669
Epoch: 4, test loss: 0.5895, test acc: 0.6768, time: 89.05343317985535

........
 --- Epoch: 5, train loss: 0.5481, train acc: 0.7047, time: 104.35402488708496
Epoch: 5, test loss: 0.6005, test acc: 0.7004, time: 105.78962850570679

........
 --- Epoch: 6, train loss: 0.5285,

#Resnet

In [ ]:
# Dataloader

class MedDataset_new(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        # print(self.img_labels.shape)
        # print(self.img_labels)

        self.img_labels['Label'] = pd.Series(np.where(self.img_labels['Label'] == 'yes', 1, 0),
          self.img_labels.index)
       # self.im
        # print(self.img_labels)
        #print( self.img_labels.iloc[0, 2])
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, ('n'+self.img_labels.iloc[idx, 1]))
        #img_path = os.path.join(self.img_dir, (self.img_labels.iloc[idx, 1]))
        image = read_image(img_path)

       # image  = torch.cat((image, image, image), 0)
        #print(image.shape)
        # if resize:
        image = fn.resize(image,128)
        label = self.img_labels.iloc[idx, 2]

        #add contrast
        #image = fn.adjust_contrast(image, 6)
        # print(label )

        #print(image.shape)
        #plt.imshow(image[0])
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        # plt.imshow(image[0])
        # print(label)
        return image, label

In [ ]:
#dataset_path = '/content/gdrive/MyDrive/NeedleImages/'

dataset_path = '/content/gdrive/MyDrive/NeedleImages/imgs1/'
# dataset = MedDataset(os.path.join(dataset_path,'Labels.csv'), dataset_path)
dataset = MedDataset_new('/content/gdrive/MyDrive/NeedleImages/Labelsc.csv', dataset_path)
generator1 = torch.Generator().manual_seed(22)

train_set, test_set = torch.utils.data.random_split(dataset, [505,127])

trainloader = DataLoader(dataset = train_set, batch_size = 64, shuffle = True, generator=generator1)
testloader = DataLoader(dataset= test_set, batch_size = 64, shuffle = True, generator=generator1)



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

# Define the ResNet model
class ResNetBinaryClassifier(nn.Module):
    def __init__(self, num_classes):
        super(ResNetBinaryClassifier, self).__init__()
        self.resnet = resnet50(pretrained=True)
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        return self.resnet(x)




In [ ]:
model = ResNetBinaryClassifier(2)

losses = []
accuracies = []
epoches = 50
start = time.time()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

# Model Training...
for epoch in range(epoches):

    epoch_loss = 0
    epoch_accuracy = 0

    for X, y in trainloader:

        preds = model(X.float())
        loss = loss_fn(preds.squeeze(1), y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        accuracy = ((preds.argmax(dim=1) == y).float().mean())
        epoch_accuracy += accuracy
        epoch_loss += loss
        print('.', end='', flush=True)

    epoch_accuracy = epoch_accuracy/len(trainloader)
    accuracies.append(epoch_accuracy)
    epoch_loss = epoch_loss / len(trainloader)
    losses.append(epoch_loss)

    print("\n --- Epoch: {}, train loss: {:.4f}, train acc: {:.4f}, time: {}".format(epoch, epoch_loss, epoch_accuracy, time.time() - start))

    # test set accuracy
    with torch.no_grad():

        test_epoch_loss = 0
        test_epoch_accuracy = 0

        for test_X, test_y in testloader:

            test_preds = model(test_X.float())
            test_loss = loss_fn(test_preds, test_y)

            test_epoch_loss += test_loss
            test_accuracy = ((test_preds.argmax(dim=1) == test_y).float().mean())
            test_epoch_accuracy += test_accuracy

        test_epoch_accuracy = test_epoch_accuracy/len(testloader)
        test_epoch_loss = test_epoch_loss / len(testloader)

        print("Epoch: {}, test loss: {:.4f}, test acc: {:.4f}, time: {}\n".format(epoch, test_epoch_loss, test_epoch_accuracy, time.time() - start))

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


........
 --- Epoch: 0, train loss: 0.8183, train acc: 0.6627, time: 342.83852887153625
Epoch: 0, test loss: 0.8433, test acc: 0.6215, time: 458.99147486686707

........
 --- Epoch: 1, train loss: 0.6198, train acc: 0.7601, time: 588.4147820472717
Epoch: 1, test loss: 0.4630, test acc: 0.7407, time: 600.9250376224518

........
 --- Epoch: 2, train loss: 0.3258, train acc: 0.8484, time: 734.5288553237915
Epoch: 2, test loss: 0.6019, test acc: 0.7558, time: 746.3216066360474

........
 --- Epoch: 3, train loss: 0.1661, train acc: 0.9351, time: 888.0674004554749
Epoch: 3, test loss: 0.6097, test acc: 0.7795, time: 900.9041900634766



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

# Define the ResNet model
class ResNetBinaryClassifier(nn.Module):
    def __init__(self, num_classes):
        super(ResNetBinaryClassifier, self).__init__()
        self.resnet = resnet50(pretrained=False)
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        return self.resnet(x)




In [ ]:
model = ResNetBinaryClassifier(2)

losses = []
accuracies = []
epoches = 50
start = time.time()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

# Model Training...
for epoch in range(epoches):

    epoch_loss = 0
    epoch_accuracy = 0

    for X, y in trainloader:

        preds = model(X.float())
        loss = loss_fn(preds.squeeze(1), y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        accuracy = ((preds.argmax(dim=1) == y).float().mean())
        epoch_accuracy += accuracy
        epoch_loss += loss
        print('.', end='', flush=True)

    epoch_accuracy = epoch_accuracy/len(trainloader)
    accuracies.append(epoch_accuracy)
    epoch_loss = epoch_loss / len(trainloader)
    losses.append(epoch_loss)

    print("\n --- Epoch: {}, train loss: {:.4f}, train acc: {:.4f}, time: {}".format(epoch, epoch_loss, epoch_accuracy, time.time() - start))

    # test set accuracy
    with torch.no_grad():

        test_epoch_loss = 0
        test_epoch_accuracy = 0

        for test_X, test_y in testloader:

            test_preds = model(test_X.float())
            test_loss = loss_fn(test_preds, test_y)

            test_epoch_loss += test_loss
            test_accuracy = ((test_preds.argmax(dim=1) == test_y).float().mean())
            test_epoch_accuracy += test_accuracy

        test_epoch_accuracy = test_epoch_accuracy/len(testloader)
        test_epoch_loss = test_epoch_loss / len(testloader)

        print("Epoch: {}, test loss: {:.4f}, test acc: {:.4f}, time: {}\n".format(epoch, test_epoch_loss, test_epoch_accuracy, time.time() - start))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialia

........
 --- Epoch: 0, train loss: 1.1056, train acc: 0.5753, time: 302.94943404197693
Epoch: 0, test loss: 0.6258, test acc: 0.6927, time: 355.4992220401764

........
 --- Epoch: 1, train loss: 0.6877, train acc: 0.6605, time: 503.01544737815857
Epoch: 1, test loss: 0.5364, test acc: 0.7083, time: 516.3734545707703

........
 --- Epoch: 2, train loss: 0.5306, train acc: 0.7525, time: 663.6416437625885
Epoch: 2, test loss: 0.5459, test acc: 0.7480, time: 676.6102097034454

........
 --- Epoch: 3, train loss: 0.3618, train acc: 0.8382, time: 841.5610182285309
Epoch: 3, test loss: 0.5435, test acc: 0.7715, time: 855.1657197475433



#CNN+Attention

In [ ]:
import torch
import torch.nn as nn

class AttentionNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_heads, num_classes):
        super(AttentionNetwork, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 16, kernel_size=(5, 5), )
        self.attention = nn.MultiheadAttention(15376, num_heads)
        self.fc = nn.Linear(16, hidden_dim)
        self.relu = nn.ReLU()
        self.output = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        # Reshape the input image
        #x = x.view(x.size(0), x.size(1), 128*128)  # (batch_size, input_dim, num_pixels)
        x = self.conv1(x)
        print(x.shape)
        x = x.view(x.size(0), x.size(1), -1)
        # Apply multi-head attention
        x, _ = self.attention(x, x, x)

        # Average pooling along the sequence dimension
        x = torch.mean(x, dim=2)

        # Apply fully connected layer
        x = self.fc(x)

        # Apply non-linearity
        x = self.relu(x)

        # Apply output layer
        x = self.output(x)

        return x


In [ ]:
model = AttentionNetwork(1,16, 8,2)

losses = []
accuracies = []
epoches = 50
start = time.time()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

# Model Training...
for epoch in range(epoches):

    epoch_loss = 0
    epoch_accuracy = 0

    for X, y in trainloader:

        preds = model(X.float())
        loss = loss_fn(preds.squeeze(1), y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        accuracy = ((preds.argmax(dim=1) == y).float().mean())
        epoch_accuracy += accuracy
        epoch_loss += loss
        print('.', end='', flush=True)

    epoch_accuracy = epoch_accuracy/len(trainloader)
    accuracies.append(epoch_accuracy)
    epoch_loss = epoch_loss / len(trainloader)
    losses.append(epoch_loss)

    print("\n --- Epoch: {}, train loss: {:.4f}, train acc: {:.4f}, time: {}".format(epoch, epoch_loss, epoch_accuracy, time.time() - start))

    # test set accuracy
    with torch.no_grad():

        test_epoch_loss = 0
        test_epoch_accuracy = 0

        for test_X, test_y in testloader:

            test_preds = model(test_X.float())
            test_loss = loss_fn(test_preds, test_y)

            test_epoch_loss += test_loss
            test_accuracy = ((test_preds.argmax(dim=1) == test_y).float().mean())
            test_epoch_accuracy += test_accuracy

        test_epoch_accuracy = test_epoch_accuracy/len(testloader)
        test_epoch_loss = test_epoch_loss / len(testloader)

        print("Epoch: {}, test loss: {:.4f}, test acc: {:.4f}, time: {}\n".format(epoch, test_epoch_loss, test_epoch_accuracy, time.time() - start))

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


torch.Size([64, 16, 124, 124])


In [ ]:
from torch.nn import Transformer
import torch.nn as nn
class AlexNet(nn.Module):
    def __init__(self, num_classes=2):
        super(AlexNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 16, kernel_size=(5, 5), stride=2, padding=1)
        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size=(5, 5), stride=2, padding=1)
        self.conv3 = nn.Conv2d(in_channels = 16, out_channels = 64, kernel_size=(3, 3), padding=1)

        # conected layers
        self.fc1 = nn.Linear(in_features= 14400, out_features=512)
        # self.fc2 = nn.Linear(in_features=512, out_features=50)
        # self.fc3 = nn.Linear(in_features=50, out_features=2)

        self.attention = nn.Sequential(
            nn.Linear(512, 128),
           # nn.Linear(256 * 7 * 7, 128),
            nn.ReLU(),
            nn.Linear(128, 2),
            nn.Sigmoid()
        )


    def forward(self, x):
        X = F.relu(self.conv1(x))
        X = F.max_pool2d(X, 2)

        # X = F.relu(self.conv2(X))
        # X = F.max_pool2d(X, 2)

        X = F.relu(self.conv3(X))
        X = F.max_pool2d(X, 2)

        X = X.view(X.shape[0], -1)
        X = F.relu(self.fc1(X))
         #print(X.shape)
        X = self.attention(X)
        # X = F.relu(self.fc2(X))

        # X = self.fc3(X)

        return X

In [ ]:
model = AlexNet()

losses = []
accuracies = []
epoches = 50
start = time.time()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

# Model Training...
for epoch in range(epoches):

    epoch_loss = 0
    epoch_accuracy = 0

    for X, y in trainloader:

        preds = model(X.float())
        loss = loss_fn(preds.squeeze(1), y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        accuracy = ((preds.argmax(dim=1) == y).float().mean())
        epoch_accuracy += accuracy
        epoch_loss += loss
        print('.', end='', flush=True)

    epoch_accuracy = epoch_accuracy/len(trainloader)
    accuracies.append(epoch_accuracy)
    epoch_loss = epoch_loss / len(trainloader)
    losses.append(epoch_loss)

    print("\n --- Epoch: {}, train loss: {:.4f}, train acc: {:.4f}, time: {}".format(epoch, epoch_loss, epoch_accuracy, time.time() - start))

    # test set accuracy
    with torch.no_grad():

        test_epoch_loss = 0
        test_epoch_accuracy = 0

        for test_X, test_y in testloader:

            test_preds = model(test_X.float())
            test_loss = loss_fn(test_preds, test_y)

            test_epoch_loss += test_loss
            test_accuracy = ((test_preds.argmax(dim=1) == test_y).float().mean())
            test_epoch_accuracy += test_accuracy

        test_epoch_accuracy = test_epoch_accuracy/len(testloader)
        test_epoch_loss = test_epoch_loss / len(testloader)

        print("Epoch: {}, test loss: {:.4f}, test acc: {:.4f}, time: {}\n".format(epoch, test_epoch_loss, test_epoch_accuracy, time.time() - start))

........
 --- Epoch: 0, train loss: 0.6381, train acc: 0.6678, time: 6.035770654678345
Epoch: 0, test loss: 0.6518, test acc: 0.6615, time: 6.883571624755859

........
 --- Epoch: 1, train loss: 0.6059, train acc: 0.7073, time: 11.843963146209717
Epoch: 1, test loss: 0.6517, test acc: 0.6616, time: 12.468335628509521

........
 --- Epoch: 2, train loss: 0.6066, train acc: 0.7066, time: 16.647462844848633
Epoch: 2, test loss: 0.6509, test acc: 0.6623, time: 17.26731562614441

........
 --- Epoch: 3, train loss: 0.6071, train acc: 0.7061, time: 21.858338594436646
Epoch: 3, test loss: 0.6516, test acc: 0.6617, time: 22.6569504737854

........
 --- Epoch: 4, train loss: 0.6064, train acc: 0.7069, time: 27.211830615997314
Epoch: 4, test loss: 0.6521, test acc: 0.6612, time: 27.825807809829712

........
 --- Epoch: 5, train loss: 0.6062, train acc: 0.7071, time: 31.890145778656006
Epoch: 5, test loss: 0.6518, test acc: 0.6615, time: 32.49920153617859

........
 --- Epoch: 6, train loss: 0.60

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNAttention(nn.Module):

    def __init__(self, num_classes):
        super(CNNAttention, self).__init__()

        # convolutional layers
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)

        # max pooling layers
        self.max_pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.max_pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # attention layer
        self.attention = nn.Sequential(
            nn.Linear(262144, 128),
           # nn.Linear(256 * 7 * 7, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

        # fully connected layer
        self.fc = nn.Linear(256 * 7 * 7, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):

        # convolutional layers
        x = F.relu(self.conv1(x))
        x = self.max_pool1(x)
        x = F.relu(self.conv2(x))
        x = self.max_pool2(x)
        x = F.relu(self.conv3(x))

        # attention layer
        attention_map = self.attention(x.view(x.size(0), -1))
        attention_map = attention_map.view(x.size(0), 1, 7, 7)
        x = attention_map * x

        # fully connected layer
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = self.softmax(x)


        return x

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        return out


class CNNAttention(nn.Module):
    def __init__(self, num_classes, attention_dim=64):
        super(CNNAttention, self).__init__()
        self.conv1 = ConvBlock(1, 64)
        self.conv2 = ConvBlock(64, 128)
        self.conv3 = ConvBlock(128, 256)
        self.attention = nn.Linear(256, attention_dim)
        self.fc = nn.Linear(256 * attention_dim, num_classes)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)

        # Apply attention
        batch_size, num_channels, height, width = out.size()
        out = out.view(batch_size, num_channels, -1).mean(dim=2)
        attention_weights = F.softmax(self.attention(out), dim=1)
        attention_out = attention_weights.unsqueeze(2).expand_as(out) * out
        attention_out = attention_out.view(batch_size, -1)

        # Final classification
        out = self.fc(attention_out)
        return out


# Usage example
# model = CNNWithAttention(num_classes=2)
# input_tensor = torch.randn(16, 1, 512, 512)  # Example input tensor of shape (batch_size, channels, height, width)
# output = model(input_tensor)
# print(output.shape)  # (16, 2) - example output shape for binary classification


In [ ]:
class CNNAttention(nn.Module):
    def __init__(self, num_classes, attention_dim=64):
        super(CNNAttention, self).__init__()
        self.conv1 = ConvBlock(1, 64)
        self.conv2 = ConvBlock(64, 128)
        self.conv3 = ConvBlock(128, 256)
        self.attention = nn.Linear(256, attention_dim)
        self.fc = nn.Linear(256 * attention_dim, num_classes)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)

        # Apply attention
        batch_size, num_channels, height, width = out.size()
        out = out.view(batch_size, num_channels, -1).mean(dim=2)
        attention_weights = F.softmax(self.attention(out), dim=1)
        attention_weights = attention_weights.unsqueeze(2).unsqueeze(3)
        attention_out = attention_weights.expand_as(out) * out
        attention_out = attention_out.view(batch_size, -1)

        # Final classification
        out = self.fc(attention_out)
        return out


In [ ]:
class CNNAttention(nn.Module):
    def __init__(self, num_classes, attention_dim=64):
        super(CNNAttention, self).__init__()
        self.conv1 = ConvBlock(1, 64)
        self.conv2 = ConvBlock(64, 128)
        self.conv3 = ConvBlock(128, 256)
        self.attention = nn.Linear(256, attention_dim)
        self.fc = nn.Linear(256 * attention_dim, num_classes)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)

        # Apply attention
        batch_size, num_channels, height, width = out.size()
        out = out.view(batch_size, num_channels, -1).mean(dim=2)
        attention_weights = F.softmax(self.attention(out), dim=1)
        attention_weights = attention_weights.unsqueeze(2).unsqueeze(3)
        attention_out = attention_weights.expand(batch_size, attention_weights.size(1), height, width) * out
        attention_out = attention_out.view(batch_size, -1)

        # Final classification
        out = self.fc(attention_out)
        return out


In [ ]:
model = CNNAttention(1)

losses = []
accuracies = []
epoches = 50
start = time.time()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

# Model Training...
for epoch in range(epoches):

    epoch_loss = 0
    epoch_accuracy = 0

    for X, y in trainloader:

        preds = model(X.float())
        loss = loss_fn(preds.squeeze(1), y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        accuracy = ((preds.argmax(dim=1) == y).float().mean())
        epoch_accuracy += accuracy
        epoch_loss += loss
        print('.', end='', flush=True)

    epoch_accuracy = epoch_accuracy/len(trainloader)
    accuracies.append(epoch_accuracy)
    epoch_loss = epoch_loss / len(trainloader)
    losses.append(epoch_loss)

    print("\n --- Epoch: {}, train loss: {:.4f}, train acc: {:.4f}, time: {}".format(epoch, epoch_loss, epoch_accuracy, time.time() - start))

    # test set accuracy
    with torch.no_grad():

        test_epoch_loss = 0
        test_epoch_accuracy = 0

        for test_X, test_y in testloader:

            test_preds = model(test_X.float())
            test_loss = loss_fn(test_preds, test_y)

            test_epoch_loss += test_loss
            test_accuracy = ((test_preds.argmax(dim=1) == test_y).float().mean())
            test_epoch_accuracy += test_accuracy

        test_epoch_accuracy = test_epoch_accuracy/len(testloader)
        test_epoch_loss = test_epoch_loss / len(testloader)

        print("Epoch: {}, test loss: {:.4f}, test acc: {:.4f}, time: {}\n".format(epoch, test_epoch_loss, test_epoch_accuracy, time.time() - start))

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


RuntimeError: ignored

#CNN+Vision transformer

In [ ]:
from torch.nn import Transformer
import torch.nn as nn
class AlexNet(nn.Module):
    def __init__(self, num_classes=2):
        super(AlexNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 16, kernel_size=(5, 5), stride=2, padding=1)
        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size=(5, 5), stride=2, padding=1)
        self.conv3 = nn.Conv2d(in_channels = 16, out_channels = 64, kernel_size=(3, 3), padding=1)

        # conected layers
        self.fc1 = nn.Linear(in_features= 14400, out_features=512)
        self.fc2 = nn.Linear(in_features=512, out_features=50)
        self.fc3 = nn.Linear(in_features=50, out_features=2)

        self.transformer = Transformer(
            d_model=512,
            nhead=8,
            num_encoder_layers=6,
            num_decoder_layers=6,
            dim_feedforward=2048,
            dropout=0.1
        )


    def forward(self, x):
        X = F.relu(self.conv1(x))
        X = F.max_pool2d(X, 2)

        # X = F.relu(self.conv2(X))
        # X = F.max_pool2d(X, 2)

        X = F.relu(self.conv3(X))
        X = F.max_pool2d(X, 2)

        X = X.view(X.shape[0], -1)
        X = F.relu(self.fc1(X))
         #print(X.shape)
        X = self.transformer(X,X)
        X = F.relu(self.fc2(X))

        X = self.fc3(X)

        return X

In [ ]:
model = AlexNet()

losses = []
accuracies = []
epoches = 50
start = time.time()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

# Model Training...
for epoch in range(epoches):

    epoch_loss = 0
    epoch_accuracy = 0

    for X, y in trainloader:

        preds = model(X.float())
        loss = loss_fn(preds.squeeze(1), y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        accuracy = ((preds.argmax(dim=1) == y).float().mean())
        epoch_accuracy += accuracy
        epoch_loss += loss
        print('.', end='', flush=True)

    epoch_accuracy = epoch_accuracy/len(trainloader)
    accuracies.append(epoch_accuracy)
    epoch_loss = epoch_loss / len(trainloader)
    losses.append(epoch_loss)

    print("\n --- Epoch: {}, train loss: {:.4f}, train acc: {:.4f}, time: {}".format(epoch, epoch_loss, epoch_accuracy, time.time() - start))

    # test set accuracy
    with torch.no_grad():

        test_epoch_loss = 0
        test_epoch_accuracy = 0

        for test_X, test_y in testloader:

            test_preds = model(test_X.float())
            test_loss = loss_fn(test_preds, test_y)

            test_epoch_loss += test_loss
            test_accuracy = ((test_preds.argmax(dim=1) == test_y).float().mean())
            test_epoch_accuracy += test_accuracy

        test_epoch_accuracy = test_epoch_accuracy/len(testloader)
        test_epoch_loss = test_epoch_loss / len(testloader)

        print("Epoch: {}, test loss: {:.4f}, test acc: {:.4f}, time: {}\n".format(epoch, test_epoch_loss, test_epoch_accuracy, time.time() - start))

........
 --- Epoch: 0, train loss: 0.6973, train acc: 0.5865, time: 14.244994878768921
Epoch: 0, test loss: 0.6317, test acc: 0.7005, time: 16.226632356643677

........
 --- Epoch: 1, train loss: 0.6265, train acc: 0.6954, time: 27.537384510040283
Epoch: 1, test loss: 0.6175, test acc: 0.7004, time: 28.9745512008667

........
 --- Epoch: 2, train loss: 0.6209, train acc: 0.6971, time: 40.17927312850952
Epoch: 2, test loss: 0.6110, test acc: 0.7015, time: 41.43684434890747

........
 --- Epoch: 3, train loss: 0.6144, train acc: 0.6981, time: 52.5534348487854
Epoch: 3, test loss: 0.6106, test acc: 0.7004, time: 53.528202295303345

........
 --- Epoch: 4, train loss: 0.6157, train acc: 0.6976, time: 64.53833246231079
Epoch: 4, test loss: 0.6125, test acc: 0.7008, time: 65.46802639961243

........
 --- Epoch: 5, train loss: 0.6145, train acc: 0.6964, time: 78.18585443496704
Epoch: 5, test loss: 0.6112, test acc: 0.7009, time: 79.1193950176239

........
 --- Epoch: 6, train loss: 0.6146, t

In [ ]:
class CNNVisionTransformer(nn.Module):
    def __init__(self):
        super(CNNVisionTransformer, self).__init__()

        # Convert 1 channel to 3 channels
        self.conv1 = nn.Conv2d(1, 3, kernel_size=1)

        # CNN backbone
        self.cnn = resnet18(pretrained=True)
        self.cnn.fc = nn.Identity()

        # Vision Transformer
        self.transformer = Transformer(
            d_model=512,
            nhead=8,
            num_encoder_layers=6,
            num_decoder_layers=6,
            dim_feedforward=2048,
            dropout=0.1
        )

        # Output layer
        self.fc = nn.Linear(512, 1)
        self.sigmoid = nn.sigmoid()

    def forward(self, x):
        # Convert 1 channel to 3 channels
        x = self.conv1(x)

        # CNN backbone
        cnn_features = self.cnn(x)

        # Vision Transformer
        transformer_output = self.transformer(cnn_features, cnn_features)  # Pass same input as source and target

        # Apply global average pooling
        # pooled_features = F.adaptive_avg_pool2d(transformer_output, 1)
        # pooled_features = pooled_features.view(pooled_features.size(0), -1)

        # Output layer
        output = self.fc(transformer_output)  # Apply sigmoid activation and squeeze the output
        output = self.sigmoid(output)

        return output

In [ ]:
#model = CNNAttention(1)

losses = []
accuracies = []
epoches = 50
start = time.time()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

# Model Training...
for epoch in range(epoches):

    epoch_loss = 0
    epoch_accuracy = 0

    for X, y in trainloader:
        # print(X.shape)
        preds = model(X.float())
        # print(preds.size())
        # print(preds)
        loss = loss_fn(preds.squeeze(1).float(), y.float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        accuracy = ((preds.argmax(dim=1) == y).float().mean())
        epoch_accuracy += accuracy
        epoch_loss += loss
        print('.', end='', flush=True)

    epoch_accuracy = epoch_accuracy/len(trainloader)
    accuracies.append(epoch_accuracy)
    epoch_loss = epoch_loss / len(trainloader)
    losses.append(epoch_loss)

    print("\n --- Epoch: {}, train loss: {:.4f}, train acc: {:.4f}, time: {}".format(epoch, epoch_loss, epoch_accuracy, time.time() - start))

    # test set accuracy
    with torch.no_grad():

        test_epoch_loss = 0
        test_epoch_accuracy = 0

        for test_X, test_y in testloader:

            test_preds = model(test_X.float())
            #print(test_preds.shape, test_y.shape)
            test_loss = loss_fn(test_preds.squeeze().float(), test_y.float())

            test_epoch_loss += test_loss
            test_accuracy = ((test_preds.argmax(dim=1) == test_y).float().mean())
            test_epoch_accuracy += test_accuracy

        test_epoch_accuracy = test_epoch_accuracy/len(testloader)
        test_epoch_loss = test_epoch_loss / len(testloader)

        print("Epoch: {}, test loss: {:.4f}, test acc: {:.4f}, time: {}\n".format(epoch, test_epoch_loss, test_epoch_accuracy, time.time() - start))

........
 --- Epoch: 0, train loss: 185.9907, train acc: 0.2856, time: 63.23578071594238
Epoch: 0, test loss: 165.5942, test acc: 0.3702, time: 67.52071404457092

........
 --- Epoch: 1, train loss: 185.7078, train acc: 0.2848, time: 123.21629071235657
Epoch: 1, test loss: 165.1619, test acc: 0.3702, time: 127.55704617500305

........
 --- Epoch: 2, train loss: 185.6349, train acc: 0.2851, time: 186.6211929321289
Epoch: 2, test loss: 164.7935, test acc: 0.3694, time: 191.8702073097229

........
 --- Epoch: 3, train loss: 185.7292, train acc: 0.2856, time: 248.0689995288849
Epoch: 3, test loss: 165.2690, test acc: 0.3709, time: 255.41748523712158

......

KeyboardInterrupt: ignored